In [1]:
#使用ResNet18初始化，使用自定义模型训练（多次多头注意力迭代）
#更改Wx输出后的大小
#

In [2]:
import matplotlib.pyplot as plt
import math
from torch.cuda.amp import autocast 
from torch.cuda.amp import GradScaler
import torch
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch import optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
import torch.nn as nn
import torchvision
from torchsummary import summary
import time
from torch.nn import init
from typing import Union, List, Dict, Any, Optional, cast
import warnings
warnings.filterwarnings("ignore")

In [3]:
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

In [4]:
#保存训练数据和模型
data_csv_path = "D:\\OneModel\\KANet.csv"   #修改此处文件名 
model_save_path = "D:\\OneModel\\KANet.pkl"  #修改此处文件名

In [5]:
train_path = "D:\\Dataset\\RAF-DB\\train"
val_path = "D:\\Dataset\\RAF-DB\\test"
#模型批次大小
batch_size = 128
resume = True
#动态学习率，学习率和循环次数增加
lr = 5e-4
epochs = 200
D_epoch = 0 
best_acc  = 0
print("epochs:",epochs,"learning_rate:",lr,"batch_size:",batch_size)

epochs: 200 learning_rate: 0.0005 batch_size: 128


In [6]:
#设备选取
flag = torch.cuda.is_available()
if flag:
    print("GPU")
else:
    print("CPU")
ngpu = 1
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
# device = torch.device("cpu")
#查看显卡名称
#torch.cuda.get_device_name()
print("divice is ", device)

#数据预处理（建议提前resize，减少每次资源的损失）放大到112x112 ，随机水平翻转
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

#增加不同种transform，预测集中去除随机翻转
val_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])

GPU
divice is  cuda:0


In [7]:
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1,bias=False)

class ECAAttention(nn.Module):

    def __init__(self, kernel_size=7):
        super().__init__()
        self.gap=nn.AdaptiveAvgPool2d(1)
        self.maxpool=nn.AdaptiveMaxPool2d(1)
        self.conv=nn.Conv1d(1,1,kernel_size=kernel_size,padding=3)
        self.sigmoid=nn.Sigmoid()
        self.init_weights()
    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                init.normal_(m.weight, std=0.001)
                if m.bias is not None:
                    init.constant_(m.bias, 0)

    def forward(self, x):
        avp_result =self.gap(x) #bs,c,1,1
        max_result = self.maxpool(x)
        
        avp_result=avp_result.squeeze(-1).permute(0,2,1) #bs,1,c
        max_result=max_result.squeeze(-1).permute(0,2,1) #bs,1,c
        
        avp_result=self.conv(avp_result) #bs,1,c
        max_result=self.conv(max_result) #bs,1,c
        
        y=self.sigmoid(max_result + avp_result) #bs,1,c
        y=y.permute(0,2,1).unsqueeze(-1) #bs,c,1,1
        return x*y.expand_as(x)

class SpatialAttention(nn.Module):
    def __init__(self,kernel_size=7):
        super().__init__()
        self.conv=nn.Conv2d(2,1,kernel_size=kernel_size,padding=3)
        self.sigmoid=nn.Sigmoid()
        self.init_weights()
    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                init.normal_(m.weight, std=0.001)
                if m.bias is not None:
                    init.constant_(m.bias, 0)
    def forward(self, x) :
        max_result,_=torch.max(x,dim=1,keepdim=True)
        avg_result=torch.mean(x,dim=1,keepdim=True)
        result=torch.cat([max_result,avg_result],1)
        output=self.conv(result)
        output=self.sigmoid(output)
        return x*output
    
class CA_SA(nn.Module):
    def __init__(self):
        super().__init__()
        self.eca = ECAAttention()
        self.sa = SpatialAttention()
        
    def forward(self,x):
        x = self.eca(x)
        x = self.sa(x)
        return x
    
class K_Attention(nn.Module):
    def __init__(self):
        super().__init__()
        self.eca = ECAAttention()
        self.sa = SpatialAttention()
        self.eca_sa = CA_SA()
    def forward(self,x):
        out1 = self.eca_sa(x)
        out2 = self.eca(x) + self.sa(x)
        return out2 + out1
class ECABasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, k_size=3):
        super(ECABasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes, 1)
        self.bn2 = nn.BatchNorm2d(planes)
        
        self.ka = K_Attention()
        
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        
        out = self.ka(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out



class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=7, k_size=[3, 5, 7, 7]):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0], int(k_size[0]))
        self.layer2 = self._make_layer(block, 128, layers[1], int(k_size[1]), stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], int(k_size[2]), stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], int(k_size[3]), stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, k_size, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, k_size))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, k_size=k_size))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


def eca_resnet18(num_classes=7, pretrained=False):
    """Constructs a ResNet-18 model.
    Args:
        k_size: Adaptive selection of kernel size
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        num_classes:The classes of classification
    """
    model = ResNet(ECABasicBlock, [2, 2, 2, 2], num_classes=num_classes)
    return model


In [8]:
#使用torchvision.datasets.ImageFolder读取数据集指定train和test文件夹
train_data = torchvision.datasets.ImageFolder(train_path, transform=train_transform)
#drop_last舍弃未满一个批次的数据        num_workers工作区一般设置为GPU个数的4倍
data0_train = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True,num_workers=4)
print(train_data)  #输出训练集相关
val_data = torchvision.datasets.ImageFolder(val_path, transform=val_transform)
data1_val = DataLoader(val_data, batch_size=batch_size, shuffle=True,drop_last=True,num_workers=4)
print(val_data)  #输出测试集相关

print(train_data.classes)  #根据分的文件夹的名字来确定的类别
print(train_data.class_to_idx) #按顺序为这些类别定义索引为0,1...
print()
print(val_data.classes)
print(val_data.class_to_idx)
print()


Dataset ImageFolder
    Number of datapoints: 12271
    Root location: D:\Dataset\RAF-DB\train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
Dataset ImageFolder
    Number of datapoints: 3068
    Root location: D:\Dataset\RAF-DB\test
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )
['Anger', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sadness', 'Surprise']
{'Anger': 0, 'Disgust': 1, 'Fear': 2, 'Happiness': 3, 'Neutral': 4, 'Sadness': 5, 'Surprise': 6}

['Anger', 'Disgust', 'Fear', 'Happiness', 'Neutral', 'Sadness', 'Surprise']
{'Anger':

In [9]:
#to(device)将模型加入GPU中加速计算
model = eca_resnet18().to(device)
#设置优化器

params = list(model.parameters()) 
optimizer = optim.AdamW(params, lr=lr)

# optimizer = optim.AdamW(model.parameters(), lr=lr)
#设置损失函数
criteon = nn.CrossEntropyLoss().to(device)
#余弦衰减学习率
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=80, eta_min=0)
#形如TensorFlow中的summary函数输出模型参数
summary(model, input_size=[(3, 224, 224)], batch_size=batch_size, device="cuda")
print()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [128, 64, 112, 112]           9,408
       BatchNorm2d-2        [128, 64, 112, 112]             128
              ReLU-3        [128, 64, 112, 112]               0
         MaxPool2d-4          [128, 64, 56, 56]               0
            Conv2d-5          [128, 64, 56, 56]          36,864
       BatchNorm2d-6          [128, 64, 56, 56]             128
              ReLU-7          [128, 64, 56, 56]               0
            Conv2d-8          [128, 64, 56, 56]          36,864
       BatchNorm2d-9          [128, 64, 56, 56]             128
AdaptiveAvgPool2d-10            [128, 64, 1, 1]               0
AdaptiveMaxPool2d-11            [128, 64, 1, 1]               0
           Conv1d-12               [128, 1, 64]               8
           Conv1d-13               [128, 1, 64]               8
          Sigmoid-14               [128

In [10]:
#测试函数
def evalute_(model,val_loader):
    model.eval()
    test_loss2 = 0.0
    test_corrects2 = 0.0
    number = 0
    for batchidx, (x, label) in enumerate(val_loader):
#         print(number)
    #torch.cuda.empty_cache()  #清除非必要GPU缓存，但是我建议不要在训练中使用此句，这可能会损失你相当多的时间
        number = number + 1
        x, label = x.to(device), label.to(device)
        #测试函数中加入no_grad()，如果不加会增加计算和显存
        with torch.no_grad():
            y1 = model(x)
            #虽然可以直接使用max函数，但是我建议在y1的比较重你最好使用F.softmax(y1,dim=1)，这样可能会有更好的效果，我在训练中使用了它
            _, preds1 = torch.max(F.softmax(y1,dim=1), 1)
            loss = criteon(y1, label)  
            
            test_loss2 += loss.item()*batch_size
            test_corrects2 += torch.sum(preds1 == label.data)
    #由于使用了最后一次抛弃，我不能使用全部测试集作为分母，这样会使最后的准确率变小
    test_loss1 = test_loss2 / (number*batch_size)
    test_acc1 = test_corrects2.double() / (number*batch_size)
#     print("TestDataset loss is ", test_loss1,"TestDataset accuracy is ",test_acc1)
    return test_acc1, test_loss1
print("执行结束")

执行结束


In [11]:
#关于AMP自动精度求解，我也并不是很熟悉，只能使用官方给的实例进行照葫芦画瓢。
scaler = torch.cuda.amp.GradScaler()
torch.cuda.empty_cache()
for epoch in range(D_epoch, epochs):
    time_one = time.time()                         #标记训练开始时间戳
    train_acc1 = 0.0
    train_loss1 = 0.0
    train_acc = 0.0
    train_loss = 0.0
    
    val_acc = 0.0
    val_loss = 0.0
    number = 0
    model.train()
    print("epoch:",epoch)

    for batchidx , (x ,label) in enumerate(data0_train):
        x , label = x.to(device), label.to(device)
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            y1 = model(x)
            loss = criteon(y1,label)  
        _, preds1 = torch.max(F.softmax(y1,dim=1), 1)
        #AMP优化
        scaler.scale(loss).backward()
#         loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm = 5, norm_type=2)  #梯度剪裁
        scaler.step(optimizer)
#         optimizer.step()
        scaler.update()
        train_loss1 += loss.item()*batch_size
        train_acc1 += torch.sum(preds1 == label.data).double()
        number = number + 1
    time_two = time.time()             #标记训练结束时间戳
    #输出训练一轮所需时间，用于分析对比
    print("The Model-Train-Time spent  %d min %.2f s"%((time_two-time_one)//60,(time_two-time_one)%60))
    #计算训练时候的平均损失和平均准确率
    train_loss = train_loss1 / (number*batch_size)
    train_acc = train_acc1 / (number*batch_size)
    #计算测试时候的平均损失和平均准确率
    val_acc, val_loss = evalute_(model, data1_val)
    
    train_acc = train_acc.cpu()
    val_acc = val_acc.cpu()
    print('Accuracy : Train is {} , Valid is {} ;  Loss : Train is  {} ,Valid is {}'.format(train_acc, val_acc, train_loss , val_loss))
    #如果你不需要训练以及验证的准确率和损失值，你可以注释这下面的两行，它们不是非必须的，理论上只存在于汇报和论文中
    dataframe = pd.DataFrame(columns = [epoch,train_acc,train_loss,val_acc, val_loss])
    dataframe.to_csv(data_csv_path,line_terminator="\n",mode='a',index=False,sep=',')
    if val_acc > best_acc:
        print("覆盖最好的模型...")
        best_acc = val_acc 
        checkpoint = {
            'epoch': epoch,
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'best_acc':best_acc
        }
        torch.save(checkpoint,model_save_path)
#     time_three = time.time() 
#     print("测试花费时间",time_three-time_two)
    scheduler.step()  #动态学习率更新 W
#如果你不是非必须，我建议你尽量不要使用n折交叉验证，使用数据增强可能效果更优于它。

epoch: 0
The Model-Train-Time spent  0 min 40.62 s
Accuracy : Train is 0.5023848684210526 , Valid is 0.5448369565217391 ;  Loss : Train is  1.3544145897815103 ,Valid is 1.2460989174635515
覆盖最好的模型...
epoch: 1
The Model-Train-Time spent  0 min 37.36 s
Accuracy : Train is 0.663733552631579 , Valid is 0.49728260869565216 ;  Loss : Train is  0.9317031025886535 ,Valid is 1.4991657785747363
epoch: 2
The Model-Train-Time spent  0 min 36.91 s
Accuracy : Train is 0.7296052631578948 , Valid is 0.662703804347826 ;  Loss : Train is  0.7555605361336156 ,Valid is 1.0040742081144582
覆盖最好的模型...
epoch: 3
The Model-Train-Time spent  0 min 36.93 s
Accuracy : Train is 0.7782894736842105 , Valid is 0.6650815217391304 ;  Loss : Train is  0.6264490102466784 ,Valid is 0.9329378864039546
覆盖最好的模型...
epoch: 4
The Model-Train-Time spent  0 min 38.17 s
Accuracy : Train is 0.8117598684210526 , Valid is 0.7391304347826086 ;  Loss : Train is  0.5290703277838857 ,Valid is 0.7606329982695372
覆盖最好的模型...
epoch: 5
The Mode

epoch: 43
The Model-Train-Time spent  0 min 46.07 s
Accuracy : Train is 0.9999177631578947 , Valid is 0.7938179347826086 ;  Loss : Train is  0.00027855065112051213 ,Valid is 1.8489768090455427
epoch: 44
The Model-Train-Time spent  0 min 49.00 s
Accuracy : Train is 1.0 , Valid is 0.7989130434782609 ;  Loss : Train is  0.0001167116000464088 ,Valid is 1.8264034053553706
epoch: 45
The Model-Train-Time spent  0 min 51.70 s
Accuracy : Train is 1.0 , Valid is 0.8012907608695652 ;  Loss : Train is  6.408289467033586e-05 ,Valid is 1.8617432272952537
epoch: 46
The Model-Train-Time spent  0 min 48.53 s
Accuracy : Train is 0.9998355263157894 , Valid is 0.8087635869565217 ;  Loss : Train is  0.00026444165330184136 ,Valid is 1.8966032188871633
覆盖最好的模型...
epoch: 47
The Model-Train-Time spent  0 min 47.91 s
Accuracy : Train is 1.0 , Valid is 0.7972146739130435 ;  Loss : Train is  3.882483824303276e-05 ,Valid is 1.971391164738199
epoch: 48
The Model-Train-Time spent  0 min 45.10 s
Accuracy : Train is 1

The Model-Train-Time spent  0 min 46.83 s
Accuracy : Train is 1.0 , Valid is 0.8057065217391304 ;  Loss : Train is  7.44077720140156e-09 ,Valid is 2.0984719836193584
epoch: 90
The Model-Train-Time spent  0 min 49.75 s
Accuracy : Train is 1.0 , Valid is 0.8050271739130435 ;  Loss : Train is  2.048909664154053e-09 ,Valid is 2.1102798969849297
epoch: 91
The Model-Train-Time spent  0 min 50.64 s
Accuracy : Train is 1.0 , Valid is 0.8053668478260869 ;  Loss : Train is  6.538864813352886e-09 ,Valid is 2.109786313513051
epoch: 92
The Model-Train-Time spent  0 min 47.43 s
Accuracy : Train is 1.0 , Valid is 0.8091032608695652 ;  Loss : Train is  4.382117798453883e-09 ,Valid is 2.065955524859221
epoch: 93
The Model-Train-Time spent  0 min 47.16 s
Accuracy : Train is 1.0 , Valid is 0.8077445652173912 ;  Loss : Train is  2.0979266417653936e-09 ,Valid is 2.0879360281902812
epoch: 94
The Model-Train-Time spent  0 min 44.10 s
Accuracy : Train is 1.0 , Valid is 0.8101222826086957 ;  Loss : Train is  6

Accuracy : Train is 1.0 , Valid is 0.8087635869565217 ;  Loss : Train is  1.0097497387936241e-09 ,Valid is 2.301522089087445
epoch: 136
The Model-Train-Time spent  0 min 51.41 s
Accuracy : Train is 1.0 , Valid is 0.8067255434782609 ;  Loss : Train is  1.8234315671418843e-09 ,Valid is 2.25067282759625
epoch: 137
The Model-Train-Time spent  0 min 51.66 s
Accuracy : Train is 1.0 , Valid is 0.8050271739130435 ;  Loss : Train is  3.5292223880165503e-10 ,Valid is 2.2819415797357974
epoch: 138
The Model-Train-Time spent  0 min 46.98 s
Accuracy : Train is 1.0 , Valid is 0.8050271739130435 ;  Loss : Train is  5.097765671579462e-10 ,Valid is 2.254864153654679
epoch: 139
The Model-Train-Time spent  0 min 47.05 s
Accuracy : Train is 1.0 , Valid is 0.8094429347826086 ;  Loss : Train is  6.960410820810418e-10 ,Valid is 2.249176994613979
epoch: 140
The Model-Train-Time spent  0 min 43.51 s
Accuracy : Train is 1.0 , Valid is 0.8019701086956521 ;  Loss : Train is  4.803663805911416e-10 ,Valid is 2.2611

The Model-Train-Time spent  0 min 50.57 s
Accuracy : Train is 1.0 , Valid is 0.8023097826086957 ;  Loss : Train is  1.3724753731175472e-10 ,Valid is 2.355743926504384
epoch: 183
The Model-Train-Time spent  0 min 46.62 s
Accuracy : Train is 1.0 , Valid is 0.8026494565217391 ;  Loss : Train is  1.2744414178948654e-10 ,Valid is 2.355781104253686
epoch: 184
The Model-Train-Time spent  0 min 46.32 s
Accuracy : Train is 1.0 , Valid is 0.8036684782608695 ;  Loss : Train is  2.1567470148990029e-10 ,Valid is 2.3223591731942217
epoch: 185
The Model-Train-Time spent  0 min 43.82 s
Accuracy : Train is 1.0 , Valid is 0.803328804347826 ;  Loss : Train is  2.352814925344367e-10 ,Valid is 2.2872377737708716
epoch: 186
The Model-Train-Time spent  0 min 43.43 s
Accuracy : Train is 1.0 , Valid is 0.8063858695652174 ;  Loss : Train is  3.921358208907278e-11 ,Valid is 2.3083797123121177
epoch: 187
The Model-Train-Time spent  0 min 47.00 s
Accuracy : Train is 1.0 , Valid is 0.8019701086956521 ;  Loss : Trai

In [12]:
#模型在末尾处，另一模型提交，导致时间成本增高